<a href="https://colab.research.google.com/github/DavidCastroPena/Vaswani2017/blob/main/replicatingAttentionIsAllWhatYouNeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:


# when we are using a transformer to analyze say a sentence ("the cat sat"), first what happens is that each word gets stored in a
# query: q_sat. Now each query or what i am looking to analize is compared (dot product) with its all the keys; all words have keys: k_the, k_cat,
# and k_sat. The dot product geometrically shows that if the direction of two vectors is related, this will reflect a large dot
#product, which shows that the two tokens are related

"""
qsat⋅kthe= 0.1
	​
𝑞sat⋅𝑘cat=2.1

qsat⋅kcat=1.5

"""

# "sat" is more related to cat than the

#Recall, the keys and query are linear transformation of their embeddings. Intuitively, we say that the query is a question
#per token that aims to uncover the role of the specific word in a given text. The word “question” is shorthand for
#something very precise: The query defines a direction in vector space along which relevance is measured.

# The key part is to understand Attention(Q,K,V)=softmax(QK^T/root(d_k)*V

#Key concepts:

#PyTorch tensors

#Matrix multiplication

#Softmax

#Masking

#Shape reasoning


#Setup
import torch
import torch.nn.functional as F

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


device: cuda


In [93]:

# Q,K ,V, and a mask are tensor. A tensor is a multi-dimensional array of numbers and in transformers all the meaning
#arises from learned tensor transformations and interactions

# embeddings are vectors, but to make computations or to represent sentences, instead of converting to
# vectors in a sequential way or one by one, tensors allow you to represent text in a more efficient way


# A vector is a tensor (a 1-D tensor). A matrix is a tensor (a 2-D tensor). So the real question is:

# Why do we need higher-dimensional tensors instead of just one vector at a time? Imagine you process one word at a time,
# using vectors only. Sentence: “The cat sat”.You would have to do this:

# Take embedding of "The" → vector compare it to "cat" → vector

# Compare it to "sat" → vector

# Repeat for "cat"

# Repeat for "sat"

# That’s nested loops sequential computation very slow hard to parallelize messy gradients. This is basically how early RNNs worked.

# What tensors give you: structure Tensors let you represent many things at once. Instead of:

# “One word → one vector” You represent: All words, all positions, all heads, all batches — at the same time


def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Q: (batch, heads, q_len, d_k)
    K: (batch, heads, k_len, d_k)
    V: (batch, heads, k_len, d_v)
    mask: (batch, heads, q_len, k_len) with 1 for allowed, 0 for blocked (optional)

    returns:
      out: (batch, heads, q_len, d_v)
      attn: (batch, heads, q_len, k_len)
    """
    d_k = Q.size(-1)

    # (batch, heads, q_len, k_len)

  #When you compute qverb*kj, you are asking:
  # "does token k, in its current representation, lie in a directaion that is useful for a verb right now?"
  # if the answer is yes (large dot product), attention weight increases, information form that token flows

  # K defines how each token can be matched by others.
  # It is the “interface” a token exposes to incoming queries.
  # A learned projection that determines under what kinds of queries this token should become relevant.


    scores = Q @ K.transpose(-2, -1) / (d_k ** 0.5)

    if mask is not None:
        # Set blocked positions to a large negative value so softmax ~ 0 there.
        scores = scores.masked_fill(mask == 0, -1e9)

#the softmax ensure converting the raw scores into probabilities of how related or important are
#the tokens to each other. However, it does not have the information itself: recall that q
#contains the question the word asks about its role, the K decide how to be matched, and V decide what
# information to contribute.

# Consider the word "bank". For matching, you may want to match syntactic role or entity type.
# For content, you may want to pass financial or river meaning. In this case, Kj encodes:
# “If another token is looking for this kind of information, how strongly should I respond?” This response
#is continuous, context-dependet, and learned statistically

# Sentence “The cat sat.” During training, the model repeatedly observes that: verbs benefit from attending to certain tokens
# those tokens often share embedding patterns (things we humans call “nouns”)
# Over time:

  # Wk learns to map those tokens into a region of space
  # that region is useful when queried by verbs

#  learns to map those tokens into a region of space,that region is useful when queried by verbs

# But the model never learns:

# “This is a noun”

# It learns: “Tokens with embeddings like this tend to be useful when verbs query the context.” That’s a relational regularity, not a category.


    attn = F.softmax(scores, dim=-1)

# # the multiplication between the attn and v: The multiplication of attention weights by values computes a learned,
# conditional linear combination of representations — mathematically similar to a regression or mixture model —
# but embedded inside a deep, nonlinear system.


    out = attn @ V
    return out, attn



In [94]:
# Lets create a small example to see the attention at work

batch, heads, seq_len, d_k, d_v = 1, 1, 4, 3, 2

Q = torch.randn(batch, heads, seq_len, d_k, device=device)
K = torch.randn(batch, heads, seq_len, d_k, device=device)
V = torch.randn(batch, heads, seq_len, d_v, device=device)

out, attn = scaled_dot_product_attention(Q, K, V)

print("Q shape:", Q.shape)
print("attn shape:", attn.shape)
print("out shape:", out.shape)

print("\nAttention matrix (q_len x k_len):\n", attn[0,0].detach().cpu())
print("\nRow sums (should be ~1):\n", attn[0,0].sum(dim=-1).detach().cpu())


Q shape: torch.Size([1, 1, 4, 3])
attn shape: torch.Size([1, 1, 4, 4])
out shape: torch.Size([1, 1, 4, 2])

Attention matrix (q_len x k_len):
 tensor([[0.1118, 0.1236, 0.1502, 0.6144],
        [0.2187, 0.1804, 0.2324, 0.3685],
        [0.1977, 0.1973, 0.2325, 0.3726],
        [0.3713, 0.0648, 0.4710, 0.0929]])

Row sums (should be ~1):
 tensor([1.0000, 1.0000, 1.0000, 1.0000])




## 1️⃣ What we implemented first: scaled dot-product attention (pure math)

We started by implementing:

Attention(Q, K, V) = softmax(QKᵀ / √dₖ) · V

This function:

- takes **Q, K, V as inputs**
- computes similarity scores via dot products
- normalizes them with softmax
- uses them to compute a weighted sum of values

At this stage:
- Q, K, V were **dummy tensors** (random numbers)
- the goal was to understand:
  - tensor shapes
  - matrix multiplication
  - softmax
  - attention matrices
  - how information flows

This step was **intentional** and **necessary**.

👉 Key point:
> `scaled_dot_product_attention` is a **standalone mathematical operator**.
> It does NOT know where Q, K, V come from.

Think of it as the **engine** of attention.

---

## 2️⃣ The missing question: where do Q, K, V come from in a real Transformer?

In the actual Transformer paper:

- Q, K, V are **not given directly**
- They are **computed from token embeddings**

The real pipeline is:

tokens  
→ embeddings X  
→ linear projections  
→ Q, K, V  
→ attention

Mathematically:

Q = X · W_Q  
K = X · W_K  
V = X · W_V  

Where:
- X is the embedding tensor (B × T × d_model)
- W_Q, W_K, W_V are **learned weight matrices**

So Q, K, V are:
- different *views* of the same tokens
- learned automatically during training

---

## 3️⃣ Why `SelfAttentionHead` exists

The purpose of `SelfAttentionHead` is **not to introduce new math**.

Its only job is to:

1. Take token embeddings `X`
2. Create Q, K, V using learned linear layers
3. Call `scaled_dot_product_attention(Q, K, V)`
4. Return:
   - contextualized token representations
   - the attention matrix

Conceptually:

SelfAttentionHead(X)
    ├── computes Q = XW_Q
    ├── computes K = XW_K
    ├── computes V = XW_V
    └── calls scaled_dot_product_attention(Q, K, V)

So:

- `scaled_dot_product_attention` = the **math**
- `SelfAttentionHead` = the **first real Transformer component**

Nothing magical is happening here.

---

## 4️⃣ Why new tensor dimensions suddenly appear (the “heads” dimension)

The attention function is written to support **multi-head attention** later.

For that reason, it expects tensors shaped like:

(Batch, Heads, SequenceLength, Dimension)

Even when:
- batch = 1
- heads = 1

So inside `SelfAttentionHead`, we do:

Q = Q.unsqueeze(1)

This:
- does NOT change the meaning of Q
- only adds a placeholder dimension for heads
- allows the same function to work for 1 head or many heads

👉 This is a **software design choice**, not a conceptual change.

---

## 5️⃣ Why this step is part of the replication plan

This step corresponds to **Section 3.2.1** of the paper:
> “Scaled Dot-Product Attention”

But now implemented as:
- a **learnable module**
- not just a formula on paper

At this point in the replication, we are still in:

✅ Step 1 — Build attention correctly

We are NOT yet:
- stacking layers
- using multiple heads
- training a model
- optimizing performance

We are simply answering:

> “How does a Transformer turn embeddings into attention outputs?”

---

## 6️⃣ Key takeaway (lock this in)

Nothing new was introduced conceptually.

We only:
- connected the attention math
- to learnable parameters
- in a reusable module

If you understand:
- how scaled dot-product attention works
- and that `SelfAttentionHead` just *creates Q, K, V*

then you understand this part of the Transformer.

---

## 7️⃣ What comes next (but not yet)

Only AFTER this is fully clear do we move on to:
- masking (causal attention)
- multi-head attention
- stacking layers

We are still building the foundation.


In [95]:


import torch
import torch.nn as nn
import torch.nn.functional as F

# This class implements a SINGLE self-attention head: he SelfAttentionHead class, which inherits from nn.Module,
# represents a concrete component within a neural network. Its primary responsibility is to create the Q, K, and V tensors from the input embeddings X using learnable linear projections (Wq, Wk, Wv).
# Why it's a class (inheriting from nn.Module): Classes inheriting from nn.Module in PyTorch are designed for components that:
# Contain Learnable Parameters: The Wq, Wk, and Wv matrices are nn.Linear layers, which are learnable. nn.Module manages these parameters, making them discoverable by optimizers during training.
# Define a forward pass: The forward method encapsulates the sequence of operations (creating Q/K/V, calling scaled_dot_product_attention, handling dimensions) that this specific module performs.
# Are part of a larger graph: By being an nn.Module, it can be easily integrated into a larger neural network (like a full Transformer block), allowing for proper dependency tracking and automatic differentiation.

class SelfAttentionHead(nn.Module):

    def __init__(self, d_model: int, d_k: int, d_v: int):
        super().__init__()

# At this point, we are passing from embeddings per each token to the matrices WQ,WK, and V learned matrices.
# The learned
        # Linear projection that maps embeddings -> query vectors
        # Wq is a learned matrix of shape (d_model, d_k)
        self.Wq = nn.Linear(d_model, d_k, bias=False)

        # Linear projection that maps embeddings -> key vectors
        # Wk is a learned matrix of shape (d_model, d_k)
        self.Wk = nn.Linear(d_model, d_k, bias=False)

        # Linear projection that maps embeddings -> value vectors
        # Wv is a learned matrix of shape (d_model, d_v)
        self.Wv = nn.Linear(d_model, d_v, bias=False)
#X: (B, T, d_model) which is a fundamental way we describe tensors in deep learning, especially when working with sequences
# like text. Let's break down each part:X: This is the input tensor itself. In the context of a Transformer,
# it usually represents a batch of token embeddings.

# B (Batch Size):

# This is the number of independent sequences (e.g., sentences, phrases, or documents) that are being processed together.
# Modern neural networks, especially when using GPUs, process data in 'batches' rather than one item at a time.
#This is much more efficient because it allows for parallel computations.
# Example: If B=3, it means you're feeding three different sentences into the model simultaneously.

#T (Sequence Length / Number of Tokens):

# This is the length of each sequence in the batch. It represents how many tokens (words, sub-words, or characters,
# depending on your tokenization) are in each individual input sequence.
# Example: If T=10, it means each of your B sentences has 10 tokens.
# d_model (Model Dimension / Embedding Dimension):

# This is the dimensionality of the vector used to represent each single token. Every token (like a word) is
#converted into a numerical vector of this size. This vector is meant to capture the semantic and positional
#information of that token. Example: If d_model=512, then each token in your T length sequences is represented
#by a vector of 512 numbers.

    def forward(self, X, mask=None):
        """
        X: (B, T, d_model)
        returns:
          out:  (B, T, d_v)
          attn: (B, T, T)
        """

        # Project embeddings -> Q/K/V
        Q = self.Wq(X)  # (B, T, d_k)
        K = self.Wk(X)  # (B, T, d_k)
        V = self.Wv(X)  # (B, T, d_v)

        # Add head dimension so we can reuse the same attention function
        Q = Q.unsqueeze(1)  # (B, 1, T, d_k)
        K = K.unsqueeze(1)  # (B, 1, T, d_k)
        V = V.unsqueeze(1)  # (B, 1, T, d_v)

        out, attn = scaled_dot_product_attention(Q, K, V, mask=mask)

        # Remove head dimension (since H=1 here)
        return out.squeeze(1), attn.squeeze(1)

In [96]:
# Explaining the  make causal mask: attn[i, j] = how much token i (the query) attends to token j (the key/value).

# Because you used a causal mask, token i is only allowed to attend to tokens ≤ i (no looking ahead).

# That’s why everything above the diagonal is zero.

def make_causal_mask(B, H, T, device):
    # lower-triangular matrix: 1 = allowed, 0 = blocked
    m = torch.tril(torch.ones(T, T, device=device))
    # expand to (B, H, T, T)
    return m.view(1, 1, T, T).expand(B, H, T, T)


In [97]:


# Testing the causal masking: torch.manual_seed(0)
# Interpreting results:
# [0.5445, 0.4555, 0.0000, 0.0000]
# ~54% to token 0

# Exactly expected.

device = "cuda" if torch.cuda.is_available() else "cpu"

B, T, d_model, d_k, d_v = 1, 4, 8, 4, 4
X = torch.randn(B, T, d_model, device=device)

head = SelfAttentionHead(d_model, d_k, d_v).to(device)

mask = make_causal_mask(B, 1, T, device)  # (B, H=1, T, T)

out, attn = head(X, mask=mask)

print("attn:\n", attn[0].detach().cpu())
print("row sums:\n", attn[0].sum(dim=-1).detach().cpu())


attn:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5098, 0.4902, 0.0000, 0.0000],
        [0.3179, 0.3470, 0.3351, 0.0000],
        [0.2265, 0.2280, 0.2319, 0.3136]])
row sums:
 tensor([1., 1., 1., 1.])


In [98]:
import torch

# Define dimensions for the SelfAttentionHead
d_model = 64  # Embedding dimension
d_k = 32      # Dimension of Query and Key vectors
d_v = 32      # Dimension of Value vectors

# Instantiate the SelfAttentionHead
attention_head = SelfAttentionHead(d_model, d_k, d_v).to(device)

# Create a dummy input tensor (batch_size, sequence_length, d_model)
batch_size = 2
sequence_length = 5
X = torch.randn(batch_size, sequence_length, d_model, device=device)

# Perform a forward pass
out, attn = attention_head(X)

print("Input X shape:", X.shape)
print("Output 'out' shape:", out.shape)
print("Attention 'attn' shape:", attn.shape)

# Verify row sums of attention matrix (should be ~1)
print("\nAttention matrix row sums (first batch, first head, should be ~1):\n", attn[0].sum(dim=-1).detach().cpu())

Input X shape: torch.Size([2, 5, 64])
Output 'out' shape: torch.Size([2, 5, 32])
Attention 'attn' shape: torch.Size([2, 5, 5])

Attention matrix row sums (first batch, first head, should be ~1):
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


# 📌 NOTE — The correct mental model: architecture vs learning in the Transformer

At this point, it is important to separate **what is being built** from **what is being learned**.

The Transformer paper combines mathematics, architecture, and training in one narrative.
In this notebook, we are intentionally **separating them**, which is why it may feel like
“nothing is being learned yet”.

This note clarifies the correct mental model.

---

## 1️⃣ Three distinct layers of understanding (do not mix them)

When reading *Attention Is All You Need*, there are **three different layers** happening at once:

---

### Layer 1 — The mathematical operator (pure attention math)

This is the formula:

Attention(Q, K, V) = softmax(QKᵀ / √dₖ) · V

At this layer:
- Q, K, V are just tensors
- no corpus is required
- no learning is required
- this is pure linear algebra

This is what we implemented first with:
- dummy Q, K, V
- random tensors
- shape checks
- attention matrices

👉 Purpose: understand **how attention works mechanically**

---

### Layer 2 — The architectural module (parameterized attention)

This is where `SelfAttentionHead` appears.

At this layer:
- we introduce **learnable matrices** W_Q, W_K, W_V
- Q, K, V are computed as:
  
  Q = X · W_Q  
  K = X · W_K  
  V = X · W_V  

- X are token embeddings
- W_Q, W_K, W_V are **trainable parameters**

Important:
- these matrices are **randomly initialized**
- they are *capable* of learning
- but they have **not learned anything yet**

👉 Purpose: build the **Transformer architecture**, not train it

This corresponds to the *architecture description* in the paper (Section 3).

---

### Layer 3 — Training (where learning actually happens)

Learning only happens when we introduce:

- a dataset (toy task or real text corpus)
- a prediction objective (e.g., next-token prediction)
- a loss function
- backpropagation
- an optimizer

Only then do:
- W_Q, W_K, W_V
- embeddings
- all other parameters

begin to change in response to data.

👉 Purpose: make attention **meaningful**

This corresponds to the *training procedure* sections of the paper.

---

## 2️⃣ What we are doing right now in this notebook

We are currently **between Layer 1 and Layer 2**.

Specifically:
- we fully understand the attention math (Layer 1)
- we have built the parameterized attention module (Layer 2)
- we have NOT introduced data, loss, or optimization (Layer 3)

So it is **expected and correct** that:
- W_Q, W_K, W_V exist
- but they do not encode syntax, semantics, or grammar
- attention patterns look random

This is not a bug.
This is the correct state at this stage.

---

## 3️⃣ Why we are not using a corpus yet

We intentionally do NOT start with real text because:

- bugs in attention math are hard to debug once training is added
- shape errors become opaque
- attention feels like “magic” instead of computation

The correct replication order is:

1. Build attention
2. Verify attention
3. Build the Transformer block
4. Train on **toy tasks**
5. Only then train on real text

This is exactly the plan we are following.

---

## 4️⃣ How the paper’s wording can be misleading

When the paper says:
> “We use learned linear projections to obtain the queries, keys, and values…”

They are describing the **final trained system**, not the initial state of the code.

In code:
- those projections exist immediately
- but they only become “learned” after training

---

## 5️⃣ Key takeaway (lock this in)

> **Right now, we are building the machine.  
> Learning only begins once we add a dataset, a loss, and optimization.**

If you understand:
- how attention works mathematically
- how Q/K/V are produced architecturally
- and why learning has not started yet

then you have the correct mental model.

---

## 6️⃣ What comes next

Before touching real text, we will:
- finish attention (masking)
- add multi-head attention
- stack layers
- then train on simple toy tasks

Only after that will we introduce a real corpus.

This is the correct and disciplined way to replicate the paper.


In [99]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention (MHA)

    Input:
      X: (B, T, d_model)

    Output:
      out:  (B, T, d_model)
      attn: (B, h, T, T)   # attention matrix per head
    """
    def __init__(self, d_model: int, num_heads: int):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.h = num_heads
        self.d_k = d_model // num_heads  # dimension per head (queries/keys/values per head)

        # Big projections: X -> Q,K,V all in d_model space
        # Each contains all heads packed together.
        self.Wq = nn.Linear(d_model, d_model, bias=False)
        self.Wk = nn.Linear(d_model, d_model, bias=False)
        self.Wv = nn.Linear(d_model, d_model, bias=False)

        # Final output projection: concat(heads) -> d_model
        self.Wo = nn.Linear(d_model, d_model, bias=False)

    def forward(self, X, mask=None):
        """
        X: (B, T, d_model)

        mask: should be broadcastable to (B, h, T, T)
              - for causal masking you can build (B, 1, T, T) and it will broadcast across heads
        """
        B, T, _ = X.shape

        # 1) Project X into packed Q, K, V
        # Shapes: (B, T, d_model)
        Q = self.Wq(X)
        K = self.Wk(X)
        V = self.Wv(X)

        # 2) Reshape packed (B, T, d_model) -> (B, h, T, d_k)
        # - view splits the last dimension into (h, d_k)
        # - transpose makes heads dimension come before time
        Q = Q.view(B, T, self.h, self.d_k).transpose(1, 2)  # (B, h, T, d_k)
        K = K.view(B, T, self.h, self.d_k).transpose(1, 2)  # (B, h, T, d_k)
        V = V.view(B, T, self.h, self.d_k).transpose(1, 2)  # (B, h, T, d_k)

        # 3) Apply scaled dot-product attention per head (in parallel)
        # out:  (B, h, T, d_k)
        # attn: (B, h, T, T)
        out, attn = scaled_dot_product_attention(Q, K, V, mask=mask)

        # 4) Concatenate heads: (B, h, T, d_k) -> (B, T, d_model)
        out = out.transpose(1, 2).contiguous().view(B, T, self.d_model)

        # 5) Final linear projection back to d_model
        out = self.Wo(out)

        return out, attn


In [100]:
#Testing multihead

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

B, T, d_model, h = 1, 4, 8, 2
X = torch.randn(B, T, d_model, device=device)

mha = MultiHeadAttention(d_model=d_model, num_heads=h).to(device)

# causal mask: (B, 1, T, T) -> broadcasts to (B, h, T, T)
mask = torch.tril(torch.ones(T, T, device=device)).view(1, 1, T, T).expand(B, 1, T, T)

out, attn = mha(X, mask=mask)

print("X:", X.shape)        # (1, 4, 8)
print("out:", out.shape)    # (1, 4, 8)
print("attn:", attn.shape)  # (1, 2, 4, 4)

print("\nHead 0 attention:\n", attn[0,0].detach().cpu())
print("\nHead 0 row sums:\n", attn[0,0].sum(dim=-1).detach().cpu())

print("\nHead 1 attention:\n", attn[0,1].detach().cpu())
print("\nHead 1 row sums:\n", attn[0,1].sum(dim=-1).detach().cpu())


X: torch.Size([1, 4, 8])
out: torch.Size([1, 4, 8])
attn: torch.Size([1, 2, 4, 4])

Head 0 attention:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4702, 0.5298, 0.0000, 0.0000],
        [0.2520, 0.4920, 0.2559, 0.0000],
        [0.2779, 0.4225, 0.1195, 0.1802]])

Head 0 row sums:
 tensor([1.0000, 1.0000, 1.0000, 1.0000])

Head 1 attention:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5070, 0.4930, 0.0000, 0.0000],
        [0.1008, 0.3130, 0.5862, 0.0000],
        [0.1322, 0.2023, 0.2607, 0.4049]])

Head 1 row sums:
 tensor([1.0000, 1.0000, 1.0000, 1.0000])


In [101]:
class PositionwiseFFN(nn.Module):
    # FFN(x) = Linear(d_model -> d_ff) + ReLU + Linear(d_ff -> d_model)
    def __init__(self, d_model: int, d_ff: int):
        super().__init__()
        self.lin1 = nn.Linear(d_model, d_ff)
        self.lin2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.lin2(F.relu(self.lin1(x)))


class TransformerBlockPostNorm(nn.Module):
    """
    One Transformer decoder block (post-norm like the original paper):

      a = MHA(x)
      x = LayerNorm(x + dropout(a))
      f = FFN(x)
      x = LayerNorm(x + dropout(f))
    """
    def __init__(self, d_model: int, num_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.mha = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.ffn = PositionwiseFFN(d_model=d_model, d_ff=d_ff)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Self-attention sublayer + residual + norm
        attn_out, attn = self.mha(x, mask=mask)
        x = self.norm1(x + self.drop(attn_out))

        # FFN sublayer + residual + norm
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.drop(ffn_out))

        return x, attn


In [102]:
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

B, T, d_model, h, d_ff = 1, 4, 8, 2, 32
x = torch.randn(B, T, d_model, device=device)

block = TransformerBlockPostNorm(d_model=d_model, num_heads=h, d_ff=d_ff, dropout=0.0).to(device)

mask = torch.tril(torch.ones(T, T, device=device)).view(1, 1, T, T).expand(B, 1, T, T)

y, attn = block(x, mask=mask)

print("x:", x.shape)        # (1,4,8)
print("y:", y.shape)        # (1,4,8)
print("attn:", attn.shape)  # (1,2,4,4)
print("head0 row sums:", attn[0,0].sum(dim=-1).detach().cpu())


x: torch.Size([1, 4, 8])
y: torch.Size([1, 4, 8])
attn: torch.Size([1, 2, 4, 4])
head0 row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000])


In [103]:
# Positional Encoding

import math

def sinusoidal_positional_encoding(T: int, d_model: int, device):
    """
    Returns PE of shape (1, T, d_model) so it can be added to X: (B, T, d_model)
    """
    pe = torch.zeros(T, d_model, device=device)

    # positions: (T, 1) = [[0],[1],[2],...]
    position = torch.arange(0, T, device=device).unsqueeze(1).float()

    # div_term: (d_model/2,) controlling frequencies
    div_term = torch.exp(
        torch.arange(0, d_model, 2, device=device).float() * (-math.log(10000.0) / d_model)
    )

    # Apply sin to even indices, cos to odd indices
    pe[:, 0::2] = torch.sin(position * div_term)  # even dims
    pe[:, 1::2] = torch.cos(position * div_term)  # odd dims

    return pe.unsqueeze(0)  # (1, T, d_model)


In [104]:
# Test positional encoding

T, d_model = 4, 8
pe = sinusoidal_positional_encoding(T, d_model, device=device)

print("PE shape:", pe.shape)  # (1,4,8)
print("PE[0,0]:", pe[0,0])    # position 0
print("PE[0,1]:", pe[0,1])    # position 1 (should differ)


PE shape: torch.Size([1, 4, 8])
PE[0,0]: tensor([0., 1., 0., 1., 0., 1., 0., 1.], device='cuda:0')
PE[0,1]: tensor([0.8415, 0.5403, 0.0998, 0.9950, 0.0100, 0.9999, 0.0010, 1.0000],
       device='cuda:0')


In [105]:
# Synthetic batch generator

def make_copy_batch_simple(batch_size, seq_len, vocab_size, device):
    x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    return x, x


In [106]:
# Minimal decoder-only transformer

class TinyDecoderLM(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        # token embedding
        self.tok_emb = nn.Embedding(vocab_size, d_model)

        # stack of Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlockPostNorm(d_model=d_model, num_heads=num_heads, d_ff=d_ff, dropout=dropout)
            for _ in range(num_layers)
        ])

        # final projection to vocabulary logits
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

    def forward(self, token_ids):
        """
        token_ids: (B, T)
        returns:
          logits: (B, T, vocab_size)
          attn_last: (B, h, T, T) attention from the last block (for inspection)
        """
        B, T = token_ids.shape
        device = token_ids.device

        # embeddings: (B, T, d_model)
        x = self.tok_emb(token_ids)

        # positional encoding: (1, T, d_model) then broadcast
        pe = sinusoidal_positional_encoding(T, self.d_model, device=device)
        x = x + pe

        # causal mask: allow attending to self and past only
        # shape (B, 1, T, T) broadcasts across heads
        mask = torch.tril(torch.ones(T, T, device=device)).view(1, 1, T, T).expand(B, 1, T, T)

        attn_last = None
        for block in self.blocks:
            x, attn_last = block(x, mask=mask)

        # logits: (B, T, vocab_size)
        logits = self.lm_head(x)
        return logits, attn_last



In [107]:
# Training loop

# --- Hyperparams (small, runs fast) ---
vocab_size = 20
seq_len = 16
batch_size = 64

d_model = 64
num_heads = 4
d_ff = 256
num_layers = 2
dropout = 0.0

steps = 600
lr = 1e-3

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

model = TinyDecoderLM(
    vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    num_layers=num_layers,
    max_len=seq_len,
    dropout=dropout
).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

model.train()
for step in range(1, steps + 1):
    inp, tgt = make_copy_batch_simple(batch_size, seq_len, vocab_size, device=device)

    logits, _ = model(inp)  # (B,T,V)

    # CrossEntropyLoss expects (N, C) and target (N,)
    loss = loss_fn(logits.reshape(-1, vocab_size), tgt.reshape(-1))

    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

    if step % 100 == 0:
        print(f"step {step:4d} | loss {loss.item():.4f}")


device: cuda
step  100 | loss 0.0249
step  200 | loss 0.0093
step  300 | loss 0.0049
step  400 | loss 0.0031
step  500 | loss 0.0021
step  600 | loss 0.0016


In [108]:
# Quick Evaluation

model.eval()
with torch.no_grad():
    inp, tgt = make_copy_batch_simple(1, seq_len, vocab_size, device=device)
    logits, attn = model(inp)
    pred = torch.argmax(logits, dim=-1)

print("inp:", inp[0].tolist())
print("tgt:", tgt[0].tolist())
print("pred:", pred[0].tolist())


inp: [1, 12, 19, 14, 12, 0, 1, 11, 0, 10, 1, 6, 19, 18, 4, 18]
tgt: [1, 12, 19, 14, 12, 0, 1, 11, 0, 10, 1, 6, 19, 18, 4, 18]
pred: [1, 12, 19, 14, 12, 0, 1, 11, 0, 10, 1, 6, 19, 18, 4, 18]


In [109]:
# Inspect one head attention from the last layer

print("attn shape:", attn.shape)  # (B, h, T, T)
print("head0 attn:\n", attn[0,0].detach().cpu())
print("head0 row sums:\n", attn[0,0].sum(dim=-1).detach().cpu())



attn shape: torch.Size([1, 4, 16, 16])
head0 attn:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2543, 0.7457, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1946, 0.2332, 0.5722, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1617, 0.3091, 0.2362, 0.2930, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1206, 0.2955, 0.1115, 0.1883, 0.2840, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1662, 0.0853, 0.2552, 0.0730, 0.0957, 0.3247, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1221, 0.1248, 0.0574, 0.1535, 0.1275, 0.2566, 0.1579, 0

In [110]:
# Lets check and confirm learning accuracy
model.eval()
with torch.no_grad():
    inp, tgt = make_copy_batch_simple(64, seq_len, vocab_size, device=device)
    logits, attn = model(inp)
    pred = logits.argmax(dim=-1)
    acc = (pred == tgt).float().mean().item()

print("token accuracy:", acc)


token accuracy: 1.0


In [111]:
# Reverse task (strong long-range diagnostic)
# Input: [a, b, c, d, ...]
# Target: [... d, c, b, a]

def make_reverse_batch(batch_size, seq_len, vocab_size, device):
    x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    y = torch.flip(x, dims=[1])  # reverse along sequence dimension
    return x, y


In [112]:
# Train on reverse
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

vocab_size = 20
seq_len = 32        # increase length to stress long-range
batch_size = 128

d_model = 64
num_heads = 4
d_ff = 128
num_layers = 2
dropout = 0.0

steps = 1200
lr = 1e-3

model = TinyDecoderLM(vocab_size, d_model, num_heads, d_ff, num_layers, max_len=seq_len, dropout=dropout).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

model.train()
for step in range(1, steps + 1):
    inp, tgt = make_reverse_batch(batch_size, seq_len, vocab_size, device=device)
    logits, _ = model(inp)
    loss = loss_fn(logits.reshape(-1, vocab_size), tgt.reshape(-1))

    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

    if step % 200 == 0:
        pred = logits.argmax(dim=-1)
        acc = (pred == tgt).float().mean().item()
        print(f"step {step:4d} | loss {loss.item():.4f} | acc {acc:.3f}")


step  200 | loss 1.6864 | acc 0.469
step  400 | loss 1.5021 | acc 0.522
step  600 | loss 1.5012 | acc 0.525
step  800 | loss 1.4985 | acc 0.526
step 1000 | loss 1.5000 | acc 0.524
step 1200 | loss 1.5002 | acc 0.526


In [113]:
# Evaluate reverse accuracy
model.eval()
with torch.no_grad():
    inp, tgt = make_reverse_batch(64, seq_len, vocab_size, device=device)
    logits, attn = model(inp)
    pred = logits.argmax(dim=-1)
    acc = (pred == tgt).float().mean().item()

print("reverse token accuracy:", acc)



reverse token accuracy: 0.52587890625


In [114]:
# Inspect attention (all heads, last layer)
print("attn shape:", attn.shape)  # (B, h, T, T)

for h in range(attn.shape[1]):
    print(f"\n--- head {h} ---")
    print(attn[0, h].detach().cpu())


attn shape: torch.Size([64, 4, 32, 32])

--- head 0 ---
tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [9.4505e-01, 5.4946e-02, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [4.7913e-01, 4.4944e-01, 7.1430e-02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [1.1300e-05, 9.0552e-03, 9.7353e-01,  ..., 1.2562e-18, 0.0000e+00,
         0.0000e+00],
        [7.6057e-02, 9.0577e-01, 1.8176e-02,  ..., 2.2223e-16, 7.9260e-16,
         0.0000e+00],
        [8.6793e-01, 1.3205e-01, 2.0111e-06,  ..., 5.3885e-14, 2.6037e-11,
         5.9605e-09]])

--- head 1 ---
tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [7.6620e-01, 2.3380e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [6.9845e-02, 5.9142e-01, 3.3874e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [1.7123e-06, 3.5058e

## Why accuracy stalls around ~0.5
# Core issue

You’re using a decoder-only causal Transformer to do sequence-to-sequence reversal.

But reversal is fundamentally “look at the whole input, then emit output”. A decoder-only model with a causal mask is forced to produce output at position t using only tokens ≤ t of the input — it cannot see the future tokens that it would need to perfectly reverse.

That’s why it can’t reach 1.0 accuracy in this setup.

What your heads show

Heads 0–2 are very peaked early (strongly attend to the beginning of the prefix), and later rows often spike around a few early positions (you see huge mass on column ~2 in late rows). That’s consistent with a decoder trying to “anchor” to early context because it can’t access the full sequence.

Head 3 looks diffuse and tiny—likely a “background” head not carrying the main routing.

So: your model is doing the best it can under a causal constraint that makes perfect reversal impossible.

In [115]:
# Lets dadd a multi head that can do cross attention

class MultiHeadAttentionGeneral(nn.Module):
    """
    Works for:
      - self-attention: Q_in = K_in = V_in = X
      - cross-attention: Q_in = decoder states, K_in/V_in = encoder memory
    """
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.0):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.h = num_heads
        self.d_k = d_model // num_heads

        self.Wq = nn.Linear(d_model, d_model, bias=False)
        self.Wk = nn.Linear(d_model, d_model, bias=False)
        self.Wv = nn.Linear(d_model, d_model, bias=False)
        self.Wo = nn.Linear(d_model, d_model, bias=False)

        self.drop = nn.Dropout(dropout)

    def forward(self, Q_in, K_in, V_in, mask=None):
        """
        Q_in: (B, Tq, d_model)
        K_in: (B, Tk, d_model)
        V_in: (B, Tk, d_model)
        mask: broadcastable to (B, h, Tq, Tk)
        """
        B, Tq, _ = Q_in.shape
        Tk = K_in.shape[1]

        Q = self.Wq(Q_in)  # (B,Tq,d_model)
        K = self.Wk(K_in)  # (B,Tk,d_model)
        V = self.Wv(V_in)  # (B,Tk,d_model)

        Q = Q.view(B, Tq, self.h, self.d_k).transpose(1, 2)  # (B,h,Tq,d_k)
        K = K.view(B, Tk, self.h, self.d_k).transpose(1, 2)  # (B,h,Tk,d_k)
        V = V.view(B, Tk, self.h, self.d_k).transpose(1, 2)  # (B,h,Tk,d_k)

        out, attn = scaled_dot_product_attention(Q, K, V, mask=mask)
        out = self.drop(out)

        out = out.transpose(1, 2).contiguous().view(B, Tq, self.d_model)  # (B,Tq,d_model)
        out = self.Wo(out)
        return out, attn


In [116]:
# Build encoder-decorder

class EncoderBlockPostNorm(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttentionGeneral(d_model, num_heads, dropout=dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        # full self-attn (no causal mask)
        a, _ = self.self_attn(x, x, x, mask=None)
        x = self.norm1(x + self.drop(a))

        f = self.ffn(x)
        x = self.norm2(x + self.drop(f))
        return x


class DecoderBlockPostNorm(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttentionGeneral(d_model, num_heads, dropout=dropout)
        self.cross_attn = MultiHeadAttentionGeneral(d_model, num_heads, dropout=dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, memory, self_mask=None):
        # 1) masked self-attn in decoder
        a, self_attn = self.self_attn(x, x, x, mask=self_mask)
        x = self.norm1(x + self.drop(a))

        # 2) cross-attn: Q from decoder, K/V from encoder memory
        c, cross_attn = self.cross_attn(x, memory, memory, mask=None)
        x = self.norm2(x + self.drop(c))

        # 3) FFN
        f = self.ffn(x)
        x = self.norm3(x + self.drop(f))

        return x, self_attn, cross_attn


In [117]:
# Build the full encoder-decoder transformer

class TinyTransformerSeq2Seq(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model

        self.src_emb = nn.Embedding(vocab_size, d_model)
        self.tgt_emb = nn.Embedding(vocab_size, d_model)

        self.enc_layers = nn.ModuleList([
            EncoderBlockPostNorm(d_model, num_heads, d_ff, dropout=dropout)
            for _ in range(num_layers)
        ])

        self.dec_layers = nn.ModuleList([
            DecoderBlockPostNorm(d_model, num_heads, d_ff, dropout=dropout)
            for _ in range(num_layers)
        ])

        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

    def forward(self, src_ids, tgt_in_ids):
        """
        src_ids:    (B, S)
        tgt_in_ids: (B, T)  (teacher-forced inputs, usually BOS + target[:-1])

        returns:
          logits: (B, T, vocab_size)
          self_attn_last:  (B,h,T,T)
          cross_attn_last: (B,h,T,S)
        """
        B, S = src_ids.shape
        T = tgt_in_ids.shape[1]
        device = src_ids.device

        # embeddings + pos enc
        src = self.src_emb(src_ids) + sinusoidal_positional_encoding(S, self.d_model, device=device)  # (B,S,d_model)
        tgt = self.tgt_emb(tgt_in_ids) + sinusoidal_positional_encoding(T, self.d_model, device=device)  # (B,T,d_model)

        # encoder (no mask)
        memory = src
        for layer in self.enc_layers:
            memory = layer(memory)  # (B,S,d_model)

        # causal mask for decoder self-attn: (B,1,T,T) broadcast over heads
        self_mask = torch.tril(torch.ones(T, T, device=device)).view(1, 1, T, T).expand(B, 1, T, T)

        self_attn_last = None
        cross_attn_last = None

        x = tgt
        for layer in self.dec_layers:
            x, self_attn_last, cross_attn_last = layer(x, memory, self_mask=self_mask)

        logits = self.lm_head(x)  # (B,T,V)
        return logits, self_attn_last, cross_attn_last



In [118]:
# Create a proper seq2seq reverse batch

def make_reverse_seq2seq_batch(batch_size, seq_len, vocab_size, device, bos_id=1):
    # avoid 0/1 as payload if you want
    src = torch.randint(2, vocab_size, (batch_size, seq_len), device=device)
    tgt = torch.flip(src, dims=[1])

    bos = torch.full((batch_size, 1), bos_id, device=device, dtype=torch.long)
    tgt_in = torch.cat([bos, tgt[:, :-1]], dim=1)

    return src, tgt_in, tgt


In [119]:
# Train it (this should climb way above 0.5)
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

vocab_size = 20
seq_len = 32
batch_size = 128

d_model = 64
num_heads = 4
d_ff = 128
num_layers = 2
dropout = 0.0

steps = 1200
lr = 1e-3

model = TinyTransformerSeq2Seq(vocab_size, d_model, num_heads, d_ff, num_layers, dropout=dropout).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

model.train()
for step in range(1, steps + 1):
    src, tgt_in, tgt_out = make_reverse_seq2seq_batch(batch_size, seq_len, vocab_size, device=device)
    logits, self_attn, cross_attn = model(src, tgt_in)

    loss = loss_fn(logits.reshape(-1, vocab_size), tgt_out.reshape(-1))

    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

    if step % 200 == 0:
        pred = logits.argmax(dim=-1)
        acc = (pred == tgt_out).float().mean().item()
        print(f"step {step:4d} | loss {loss.item():.4f} | acc {acc:.3f}")


device: cuda
step  200 | loss 0.3264 | acc 0.902
step  400 | loss 0.0074 | acc 1.000
step  600 | loss 0.0021 | acc 1.000
step  800 | loss 0.0013 | acc 1.000
step 1000 | loss 0.0009 | acc 1.000
step 1200 | loss 0.0005 | acc 1.000


In [120]:
# Evaluate + inspect cross-attention

model.eval()
with torch.no_grad():
    src, tgt_in, tgt_out = make_reverse_seq2seq_batch(64, seq_len, vocab_size, device=device)
    logits, self_attn, cross_attn = model(src, tgt_in)
    pred = logits.argmax(dim=-1)
    acc = (pred == tgt_out).float().mean().item()

print("reverse seq2seq token accuracy:", acc)
print("self_attn shape:", self_attn.shape)     # (B,h,T,T)
print("cross_attn shape:", cross_attn.shape)   # (B,h,T,S)


reverse seq2seq token accuracy: 1.0
self_attn shape: torch.Size([64, 4, 32, 32])
cross_attn shape: torch.Size([64, 4, 32, 32])


## Results of replication:

A working encoder–decoder Transformer that learns reversal perfectly at seq_len=32, and your training curve (0.326 → 0.0005) is exactly what we want in a replication narrative.

## What this proves (and how it maps to the paper)

Your architecture is correct: encoder self-attn + decoder masked self-attn + cross-attn + FFN + residual + LayerNorm.

Your training loop is correct: gradients flow, Wq/Wk/Wv learn.

You’ve empirically shown the “short path length” advantage: the decoder can directly attend (via cross-attn) to any encoder position in one hop.

## Lets show the mechanism (visualize cross-attention)

For reversal, the most interesting thing is cross-attention:

Decoder position t is trying to output the token that came from encoder position S-1-t.

So you expect cross-attention to form a reversal alignment pattern.

The following evidence shows: Cross-attention is content-addressed retrieval: the decoder uses queries to retrieve relevant encoder values, and for reversal, it learns a near-deterministic alignment.

In [121]:
b = 0   # pick one example from batch
h = 0   # head index

print("src:", src[b].tolist())
print("tgt_out:", tgt_out[b].tolist())
print("\nCross-attn head 0 (T x S):\n", cross_attn[b, h].detach().cpu())

# For each decoder position t, which encoder position got max attention?
align = cross_attn[b, h].argmax(dim=-1).detach().cpu()
print("\nArgmax alignment (decoder_pos -> encoder_pos):", align.tolist())


src: [12, 14, 9, 10, 11, 13, 17, 14, 15, 11, 12, 8, 17, 9, 17, 10, 13, 2, 11, 8, 19, 13, 3, 2, 13, 14, 13, 11, 5, 16, 14, 15]
tgt_out: [15, 14, 16, 5, 11, 13, 14, 13, 2, 3, 13, 19, 8, 11, 2, 13, 10, 17, 9, 17, 8, 12, 11, 15, 14, 17, 13, 11, 10, 9, 14, 12]

Cross-attn head 0 (T x S):
 tensor([[6.9417e-09, 1.1615e-09, 8.9411e-12,  ..., 1.6311e-02, 2.2106e-01,
         7.6242e-01],
        [8.6573e-14, 8.7116e-15, 6.4544e-17,  ..., 9.7794e-02, 4.6557e-01,
         4.3030e-01],
        [6.1266e-17, 3.8531e-18, 3.4753e-20,  ..., 2.5964e-01, 4.9373e-01,
         1.8851e-01],
        ...,
        [9.9824e-03, 9.1933e-02, 2.8420e-01,  ..., 1.7112e-23, 9.3963e-22,
         1.2453e-19],
        [1.1765e-01, 4.7845e-01, 2.9743e-01,  ..., 1.3798e-17, 8.1354e-16,
         6.5235e-14],
        [1.9405e-01, 5.3661e-01, 2.1527e-01,  ..., 1.1635e-16, 7.0131e-15,
         6.0266e-13]])

Argmax alignment (decoder_pos -> encoder_pos): [31, 30, 30, 28, 27, 27, 25, 24, 23, 23, 21, 21, 20, 18, 17, 16, 15, 15

# Step 6 — Small MT (fast, end-to-end pipeline)

Goal: Move from toy tasks → a real machine translation task (German → English) using IWSLT14 de–en.
We’re not chasing SOTA yet. We’re validating the full pipeline end-to-end, then improving it.

---

## Why this sequence?
Machine translation has many moving parts: dataset loading, tokenization, vocabulary, padding/masks, teacher forcing,
training loop, decoding, and evaluation metrics. If we try to do everything at once (e.g., SentencePiece + full training),
it’s hard to debug. This staged sequence ensures each layer works before adding complexity.

---

## The fast sequence (recommended)

### 1) Load dataset (Hugging Face)
**What we do**
- Load IWSLT14 de–en from HF.
- Inspect one example to confirm the schema:
  `{"translation": {"de": "...", "en": "..."}}`

**Why**
- HF datasets are reliable and easy to reproduce in Colab.
- Verifies we have real parallel data before writing tokenization/model code.

**Success criteria**
- We can print a sample `de` and `en` sentence.
- `train`, `validation`, and `test` splits exist.

---

### 2) Build a tiny vocab (top N tokens) + `<unk>`
**What we do**
- Tokenize with a simple baseline tokenizer (whitespace split).
- Count tokens on a training slice (e.g., 20k examples).
- Build separate vocabularies for source (de) and target (en):
  - `<pad>`, `<bos>`, `<eos>`, `<unk>` + top-N tokens

**Why**
- Fast and transparent: we understand every token ↔ id mapping.
- Great for debugging padding, masks, and teacher forcing.
- This is a **baseline** tokenizer — not the final one.

**Success criteria**
- Vocab sizes match expectations (e.g., ~12k + special tokens).
- Unknown words map to `<unk>`.

---

### 3) Train for 1 epoch on a slice (≈ 20k) to verify end-to-end
**What we do**
- Use a small slice of training data to keep iteration fast.
- Implement DataLoader + padding.
- Train encoder–decoder Transformer with:
  - encoder self-attn (full)
  - decoder self-attn (causal mask)
  - decoder cross-attn (to encoder memory)
  - pad masks for encoder keys and decoder keys
  - loss ignores `<pad>` via `ignore_index=pad_id`

**Why**
- Confirms the *entire* forward pass + masking + loss + optimizer are correct.
- If loss doesn’t decrease here, we debug before adding tokenizers/long training.

**Success criteria**
- Training loss decreases over the epoch.
- Validation loss is reasonable (not NaN, not exploding).

---

### 4) Add greedy decoding + BLEU (quick evaluation)
**What we do**
- Implement greedy decoding:
  - start with `<bos>`
  - iteratively pick argmax token until `<eos>` or max_len
- Compute BLEU using sacreBLEU on a small validation subset.

**Why**
- Loss alone doesn’t guarantee translations look correct.
- BLEU gives a standardized metric for sanity checking and later comparisons.

**Success criteria**
- Decoding runs end-to-end without crashing.
- BLEU is computed successfully (it may be low early — that’s fine).
- We can print: source / reference / hypothesis.

---

### 5) Upgrade tokenizer → SentencePiece and train longer (improvement phase)
**What we do**
- Replace whitespace tokens with SentencePiece BPE.
- Rebuild vocabs and re-train.
- Increase training data (full train split) and training time.

**Why**
- SentencePiece reduces `<unk>` rate and handles morphology/subwords better.
- Typically improves BLEU substantially compared to whitespace vocab.

**Success criteria**
- BLEU improves vs baseline.
- Model produces more fluent, less `<unk>`-heavy outputs.

---

## Notes / Guardrails
- This step is about correctness + reproducibility first, then performance.
- If something fails, we reduce scope:
  - smaller vocab, shorter max_len, fewer layers, smaller data slice.
- Always verify:
  1) shapes
  2) masks (causal + pad)
  3) loss decreases on a small slice
  4) decoding works before training longer


In [122]:
!pip -q install datasets sacrebleu


In [130]:
from datasets import load_dataset

ds = load_dataset("Helsinki-NLP/tatoeba_mt", "deu-eng")  # splits: train/validation/test
print(ds)
print(ds["train"][0]["translation"])


RuntimeError: Dataset scripts are no longer supported, but found tatoeba_mt.py

In [ ]:
# Build tiny vocab (top N) with <unk>

from collections import Counter

# Special tokens
PAD, BOS, EOS, UNK = "<pad>", "<bos>", "<eos>", "<unk>"

pad_id, bos_id, eos_id, unk_id = 0, 1, 2, 3

def tokenize_basic(s: str):
    return s.strip().split()

def build_vocab(examples, lang_key, max_vocab=12000):
    counter = Counter()
    for ex in examples:
        text = ex["translation"][lang_key]
        counter.update(tokenize_basic(text))
    most_common = counter.most_common(max_vocab - 4)

    itos = [PAD, BOS, EOS, UNK] + [w for w, _ in most_common]
    stoi = {w:i for i,w in enumerate(itos)}
    return stoi, itos

# Take a slice of training for speed (20k)
train_slice = ds["train"].select(range(20000))

src_stoi, src_itos = build_vocab(train_slice, "de", max_vocab=12000)
tgt_stoi, tgt_itos = build_vocab(train_slice, "en", max_vocab=12000)

print("src vocab size:", len(src_itos))
print("tgt vocab size:", len(tgt_itos))


In [ ]:
# Numericalize + DataLoader (padding)

import torch
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

def encode(text, stoi, max_len):
    toks = tokenize_basic(text)[:max_len-2]
    ids = [bos_id] + [stoi.get(t, unk_id) for t in toks] + [eos_id]
    return ids

def collate_fn(batch, max_src_len=64, max_tgt_len=64):
    # batch: list of dicts with "translation"
    src_ids_list = []
    tgt_in_list = []
    tgt_out_list = []

    for ex in batch:
        src_text = ex["translation"]["de"]
        tgt_text = ex["translation"]["en"]

        src_ids = encode(src_text, src_stoi, max_src_len)              # [BOS ... EOS]
        tgt_ids = encode(tgt_text, tgt_stoi, max_tgt_len)              # [BOS ... EOS]

        # Teacher forcing:
        # tgt_in  = [BOS, w1, w2, ...]
        # tgt_out = [w1,  w2, ... , EOS]
        tgt_in  = tgt_ids[:-1]
        tgt_out = tgt_ids[1:]

        src_ids_list.append(src_ids)
        tgt_in_list.append(tgt_in)
        tgt_out_list.append(tgt_out)

    def pad_to_max(seqs, pad_value):
        max_len = max(len(s) for s in seqs)
        out = torch.full((len(seqs), max_len), pad_value, dtype=torch.long)
        for i,s in enumerate(seqs):
            out[i, :len(s)] = torch.tensor(s, dtype=torch.long)
        return out

    src = pad_to_max(src_ids_list, pad_id)       # (B, S)
    tgt_in = pad_to_max(tgt_in_list, pad_id)     # (B, T)
    tgt_out = pad_to_max(tgt_out_list, pad_id)   # (B, T)

    return src.to(device), tgt_in.to(device), tgt_out.to(device)

train_loader = DataLoader(train_slice, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(ds["validation"], batch_size=64, shuffle=False, collate_fn=collate_fn)


In [ ]:
# Update your model to support padding masks (important for MT)

# Your seq2seq toy model worked because there was no padding. MT batches have padding, so we must block attention to PAD tokens.
# Drop-in patch: make your decoder block accept cross_mask + self_mask
#If you already have MultiHeadAttentionGeneral, PositionwiseFFN, and your scaled_dot_product_attention,
# paste this MT-ready model (it’s basically your Option B, but with pad-masks):

import torch.nn as nn
import torch.nn.functional as F
import math

def sinusoidal_positional_encoding(T: int, d_model: int, device):
    pe = torch.zeros(T, d_model, device=device)
    position = torch.arange(0, T, device=device).unsqueeze(1).float()
    div_term = torch.exp(torch.arange(0, d_model, 2, device=device).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe.unsqueeze(0)  # (1,T,d_model)

def scaled_dot_product_attention(Q, K, V, mask=None):
    d_k = Q.size(-1)
    scores = Q @ K.transpose(-2, -1) / (d_k ** 0.5)  # (B,h,Tq,Tk)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    attn = F.softmax(scores, dim=-1)
    out = attn @ V
    return out, attn

class PositionwiseFFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class MultiHeadAttentionGeneral(nn.Module):
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.0):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.h = num_heads
        self.d_k = d_model // num_heads

        self.Wq = nn.Linear(d_model, d_model, bias=False)
        self.Wk = nn.Linear(d_model, d_model, bias=False)
        self.Wv = nn.Linear(d_model, d_model, bias=False)
        self.Wo = nn.Linear(d_model, d_model, bias=False)
        self.drop = nn.Dropout(dropout)

    def forward(self, Q_in, K_in, V_in, mask=None):
        B, Tq, _ = Q_in.shape
        Tk = K_in.shape[1]

        Q = self.Wq(Q_in)
        K = self.Wk(K_in)
        V = self.Wv(V_in)

        Q = Q.view(B, Tq, self.h, self.d_k).transpose(1, 2)  # (B,h,Tq,d_k)
        K = K.view(B, Tk, self.h, self.d_k).transpose(1, 2)  # (B,h,Tk,d_k)
        V = V.view(B, Tk, self.h, self.d_k).transpose(1, 2)  # (B,h,Tk,d_k)

        out, attn = scaled_dot_product_attention(Q, K, V, mask=mask)
        out = self.drop(out)
        out = out.transpose(1, 2).contiguous().view(B, Tq, self.d_model)
        out = self.Wo(out)
        return out, attn

class EncoderBlockPostNorm(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttentionGeneral(d_model, num_heads, dropout=dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, enc_mask=None):
        a, _ = self.self_attn(x, x, x, mask=enc_mask)
        x = self.norm1(x + self.drop(a))
        f = self.ffn(x)
        x = self.norm2(x + self.drop(f))
        return x

class DecoderBlockPostNorm(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttentionGeneral(d_model, num_heads, dropout=dropout)
        self.cross_attn = MultiHeadAttentionGeneral(d_model, num_heads, dropout=dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, memory, self_mask=None, cross_mask=None):
        a, self_attn = self.self_attn(x, x, x, mask=self_mask)
        x = self.norm1(x + self.drop(a))

        c, cross_attn = self.cross_attn(x, memory, memory, mask=cross_mask)
        x = self.norm2(x + self.drop(c))

        f = self.ffn(x)
        x = self.norm3(x + self.drop(f))
        return x, self_attn, cross_attn

class TinyTransformerSeq2SeqMT(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.src_emb = nn.Embedding(src_vocab, d_model)
        self.tgt_emb = nn.Embedding(tgt_vocab, d_model)
        self.enc_layers = nn.ModuleList([EncoderBlockPostNorm(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dec_layers = nn.ModuleList([DecoderBlockPostNorm(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.lm_head = nn.Linear(d_model, tgt_vocab, bias=False)

    def forward(self, src_ids, tgt_in_ids, src_key_mask, tgt_key_mask):
        # src_ids: (B,S), tgt_in_ids: (B,T)
        B, S = src_ids.shape
        T = tgt_in_ids.shape[1]
        device = src_ids.device

        src = self.src_emb(src_ids) + sinusoidal_positional_encoding(S, self.d_model, device=device)
        tgt = self.tgt_emb(tgt_in_ids) + sinusoidal_positional_encoding(T, self.d_model, device=device)

        # Encoder mask: block attending TO padded source positions
        # src_key_mask: (B,S) True for real tokens
        enc_mask = src_key_mask.view(B, 1, 1, S).expand(B, 1, S, S)

        memory = src
        for layer in self.enc_layers:
            memory = layer(memory, enc_mask=enc_mask)

        # Decoder self-attn mask = causal AND "don’t attend to PAD keys"
        causal = torch.tril(torch.ones(T, T, device=device)).view(1, 1, T, T)
        dec_key = tgt_key_mask.view(B, 1, 1, T).expand(B, 1, T, T)
        self_mask = causal.expand(B, 1, T, T) * dec_key

        # Cross mask: decoder queries can only attend to non-pad encoder keys
        cross_mask = src_key_mask.view(B, 1, 1, S).expand(B, 1, T, S)

        self_attn_last = None
        cross_attn_last = None
        x = tgt
        for layer in self.dec_layers:
            x, self_attn_last, cross_attn_last = layer(x, memory, self_mask=self_mask, cross_mask=cross_mask)

        logits = self.lm_head(x)
        return logits, self_attn_last, cross_attn_last



In [ ]:
# Train for 1 epoch on a 20k slice (verify end-to-end)

import torch

# Hyperparams (small but real)
d_model = 128
num_heads = 4
d_ff = 256
num_layers = 2
dropout = 0.1

model = TinyTransformerSeq2SeqMT(
    src_vocab=len(src_itos),
    tgt_vocab=len(tgt_itos),
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    num_layers=num_layers,
    dropout=dropout
).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id)

def run_epoch(loader, train=True):
    model.train() if train else model.eval()
    total_loss = 0.0
    n_batches = 0

    for src, tgt_in, tgt_out in loader:
        src_key_mask = (src != pad_id)      # (B,S)
        tgt_key_mask = (tgt_in != pad_id)   # (B,T)

        logits, _, _ = model(src, tgt_in, src_key_mask, tgt_key_mask)
        loss = loss_fn(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

        if train:
            opt.zero_grad(set_to_none=True)
            loss.backward()
            opt.step()

        total_loss += loss.item()
        n_batches += 1

    return total_loss / max(1, n_batches)

# 1 epoch on train slice
train_loss = run_epoch(train_loader, train=True)
val_loss = run_epoch(val_loader, train=False)
print("train_loss:", train_loss)
print("val_loss:", val_loss)



In [ ]:
# Greedy decoding and sacreBLEU

def decode_ids(ids, itos):
    # stop at EOS, skip special tokens
    out = []
    for i in ids:
        if i == eos_id:
            break
        if i in (pad_id, bos_id):
            continue
        out.append(itos[i] if i < len(itos) else UNK)
    return " ".join(out)

@torch.no_grad()
def greedy_translate(src_ids, max_len=64):
    model.eval()
    B, S = src_ids.shape
    src_key_mask = (src_ids != pad_id)

    # start with BOS
    tgt_in = torch.full((B, 1), bos_id, dtype=torch.long, device=device)

    for _ in range(max_len):
        tgt_key_mask = (tgt_in != pad_id)
        logits, _, _ = model(src_ids, tgt_in, src_key_mask, tgt_key_mask)
        next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)  # (B,1)
        tgt_in = torch.cat([tgt_in, next_token], dim=1)
        if (next_token == eos_id).all():
            break

    return tgt_in  # includes BOS + generated tokens



In [ ]:
# Compute corpus BLUE with SacreBLEU

from sacrebleu.metrics import BLEU

bleu = BLEU()  # standardized BLEU scorer :contentReference[oaicite:3]{index=3}

@torch.no_grad()
def eval_bleu(loader, num_batches=20):
    model.eval()
    hyps = []
    refs = []

    for i, (src, tgt_in, tgt_out) in enumerate(loader):
        if i >= num_batches:
            break

        pred_ids = greedy_translate(src, max_len=64)  # (B, <=max_len)

        for b in range(src.size(0)):
            hyp = decode_ids(pred_ids[b].tolist(), tgt_itos)

            # rebuild reference string from tgt_out (shifted)
            ref = decode_ids(tgt_out[b].tolist(), tgt_itos)

            hyps.append(hyp)
            refs.append(ref)

    score = bleu.corpus_score(hyps, [refs])
    return score

bleu_score = eval_bleu(val_loader, num_batches=20)
print("BLEU on validation (subset):", bleu_score)


In [ ]:
# Show a few qualitative examples from validation
model.eval()
with torch.no_grad():
    batch = next(iter(val_loader))
    src, tgt_in, tgt_out = batch

    pred_ids = greedy_translate(src[:3], max_len=64)

    for i in range(3):
        # Source (German)
        src_text = decode_ids(src[i].tolist(), src_itos)
        # Reference (English)
        ref_text = decode_ids(tgt_out[i].tolist(), tgt_itos)
        # Hypothesis (English)
        hyp_text = decode_ids(pred_ids[i].tolist(), tgt_itos)

        print("="*80)
        print("SRC (de):", src_text)
        print("REF (en):", ref_text)
        print("HYP (en):", hyp_text)




## ⚠️ Diagnosis: Need for SentencePiece Tokenization

The current baseline tokenizer (simple whitespace splitting) has allowed us to verify the Transformer architecture and training pipeline. However, it exhibits significant limitations that hinder translation quality, particularly evident in the qualitative examples:

1.  **High `<unk>` Rate**:
    *   Many words, especially less frequent ones or inflected forms, are not present in our limited 12,000-token vocabulary derived from a small training slice. These words are replaced by the `<unk>` token.
    *   **Problem**: When a model sees `<unk>` tokens in the input or is forced to generate them in the output, it loses semantic information. A translation like "A `<unk>` strategy to counter the `<unk>` of `<unk>`" is practically meaningless because key nouns and adjectives are unknown. This severely limits the model's ability to produce accurate or fluent translations.

2.  **Inflectional and Compound Word Challenges (German Specific)**:
    *   German is a highly inflected language (e.g., different endings for cases, genders) and frequently forms long compound words (e.g., "Krankenhaus", "Donaudampfschifffahrtsgesellschaftskapitän").
    *   **Problem**: A whitespace tokenizer treats each unique inflected form or compound as a completely new word. This drastically increases vocabulary size (even with frequency-based truncation, many are still `<unk>`), and the model cannot easily learn relationships between "gehen" (to go), "geht" (goes), "gegangen" (gone). Similarly, common sub-word units within compounds (like "kranken", "haus") are lost, forcing the model to memorize entire long words.

3.  **Limited Vocabulary Size & Coverage**:
    *   Even with 12,000 tokens, a vocabulary built on raw words often struggles to cover the diversity of natural language, especially for morphologically rich languages. Many infrequent but important words will always fall outside the vocabulary.
    *   **Problem**: This directly leads to the high `<unk>` rate observed, making it impossible for the model to translate sentences containing out-of-vocabulary words accurately.

---

## ✨ Solution: SentencePiece with Byte Pair Encoding (BPE)

SentencePiece, specifically using the Byte Pair Encoding (BPE) algorithm, offers a robust solution to these problems:

1.  **Subword Tokenization**:
    *   Instead of splitting into full words, BPE breaks down rare or unknown words into smaller, frequently occurring subword units. For example, "unbelievable" might become "un", "believe", "able". In German, "Krankenhaus" could become "Kranken" and "haus".
    *   **Benefit**: This allows the model to handle unseen words by composing them from known subword units. The number of `<unk>` tokens is drastically reduced, as almost any word can be represented.

2.  **Manages Inflection and Compounding**:
    *   By breaking down words into subwords, BPE naturally handles different inflections and compound words more effectively. The model can learn the meaning of common prefixes, suffixes, and word stems.
    *   **Benefit**: This significantly improves the model's ability to generalize to new or complex word forms, especially in languages like German.

3.  **Smaller, More Representative Vocabulary**:
    *   A SentencePiece vocabulary (typically 32k or 64k tokens) consists of common words and frequently occurring subword units. This vocabulary is much more efficient than a word-level vocabulary of the same size.
    *   **Benefit**: It provides better coverage of the language with a manageable vocabulary size, allowing the model to learn more robust representations.

4.  **Language-Agnostic and Unified Processing**:
    *   SentencePiece trains directly on raw text without requiring pre-tokenized input or language-specific rules. It handles whitespace consistently, often treating it as a special character.
    *   **Benefit**: This simplifies the preprocessing pipeline and ensures consistent tokenization across different languages.

**In summary, upgrading to SentencePiece BPE is a crucial step to move beyond the limitations of simple whitespace tokenization, enabling the Transformer model to learn more effective and generalizable representations for machine translation, thereby improving overall translation quality and significantly reducing the "noise" introduced by `<unk>` tokens.**

In [ ]:
pip install -q sentencepiece

In [127]:
import sentencepiece as spm

# Load the trained SentencePiece models
sp_de = spm.SentencePieceProcessor(model_file='de_sp.model')
sp_en = spm.SentencePieceProcessor(model_file='en_sp.model')

# Define special token IDs based on SentencePiece defaults
# SentencePiece typically uses 0 for <unk>, 1 for <s>, 2 for </s>
# We will use 0 for padding as well, a common practice where padding takes the <unk> ID
pad_id = sp_en.unk_id() # Use <unk> id for padding
bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
unk_id = sp_en.unk_id()

print(f"SentencePiece: pad_id={pad_id}, bos_id={bos_id}, eos_id={eos_id}, unk_id={unk_id}")

SentencePiece: pad_id=0, bos_id=1, eos_id=2, unk_id=0


In [128]:
# New encode function using SentencePiece
def encode_spm(text, sp_model, max_len):
    # sp_model.encode_as_ids adds <s> and </s> by default if not suppressed
    ids = sp_model.encode_as_ids(text)

    # Ensure BOS and EOS are correctly handled.
    # SentencePiece encode_as_ids usually includes <s> and </s> by default.
    # We need to make sure they match our expected bos_id and eos_id consistently.
    # sp_model.encode_as_ids(text, add_bos=True, add_eos=True) is the default behavior

    # Clip sequence to max_len (including BOS/EOS if present)
    if len(ids) > max_len:
        # Preserve BOS and EOS if they are automatically added, or handle manually
        # Assuming sp_model.encode_as_ids already added them, we clip from the middle if necessary
        # For simplicity, if length exceeds max_len, we just truncate, ensuring BOS/EOS are at ends
        # A more robust solution might ensure BOS/EOS are always present after clipping
        ids = ids[:max_len-1] + [sp_model.eos_id()] # Truncate and ensure EOS
    elif len(ids) < max_len and ids[-1] != sp_model.eos_id():
        # Add EOS if not present (should be by default, but double check)
        ids.append(sp_model.eos_id())

    return ids

# Updated collate_fn to use SentencePiece
def collate_fn_spm(batch, max_src_len=64, max_tgt_len=64):
    src_ids_list = []
    tgt_in_list = []
    tgt_out_list = []

    for ex in batch:
        src_text = ex["translation"]["de"]
        tgt_text = ex["translation"]["en"]

        # Encode using SentencePiece
        src_ids = encode_spm(src_text, sp_de, max_src_len) # Contains <s>...</s>
        tgt_ids = encode_spm(tgt_text, sp_en, max_tgt_len) # Contains <s>...</s>

        # Teacher forcing:
        # tgt_in  = [BOS, w1, w2, ...]
        # tgt_out = [w1,  w2, ... , EOS]
        # SentencePiece encode_as_ids includes <s> and </s>.
        # We need to adjust for tgt_in/tgt_out if sp_model.encode_as_ids adds them.
        # Default behavior: sp_model.encode_as_ids adds <s> and </s>
        # So, src_ids will be [<s>, token_ids..., </s>]
        # tgt_ids will be [<s>, token_ids..., </s>]

        # If sp_model.encode_as_ids(text) results in [<s>, tok1, tok2, ..., tokN, </s>]
        # then for training, tgt_in should be [<s>, tok1, ..., tokN]
        # and tgt_out should be [tok1, ..., tokN, </s>]
        tgt_in  = tgt_ids[:-1] # Remove the final </s> from tgt_ids for input
        tgt_out = tgt_ids[1:]  # Remove the initial <s> from tgt_ids for target

        src_ids_list.append(src_ids)
        tgt_in_list.append(tgt_in)
        tgt_out_list.append(tgt_out)

    def pad_to_max(seqs, pad_value):
        max_len = max(len(s) for s in seqs)
        out = torch.full((len(seqs), max_len), pad_value, dtype=torch.long)
        for i,s in enumerate(seqs):
            out[i, :len(s)] = torch.tensor(s, dtype=torch.long)
        return out

    src = pad_to_max(src_ids_list, pad_id)       # (B, S)
    tgt_in = pad_to_max(tgt_in_list, pad_id)     # (B, T)
    tgt_out = pad_to_max(tgt_out_list, pad_id)   # (B, T)

    return src.to(device), tgt_in.to(device), tgt_out.to(device)

# Re-create DataLoaders with the new collate_fn_spm
train_loader_spm = DataLoader(train_slice, batch_size=64, shuffle=True, collate_fn=lambda b: collate_fn_spm(b, max_src_len=64, max_tgt_len=64))
val_loader_spm = DataLoader(ds["validation"], batch_size=64, shuffle=False, collate_fn=lambda b: collate_fn_spm(b, max_src_len=64, max_tgt_len=64))

# Get updated vocabulary sizes
src_vocab_size_spm = sp_de.get_piece_size()
tgt_vocab_size_spm = sp_en.get_piece_size()

print(f"Updated Source Vocab Size (SentencePiece): {src_vocab_size_spm}")
print(f"Updated Target Vocab Size (SentencePiece): {tgt_vocab_size_spm}")

Updated Source Vocab Size (SentencePiece): 16000
Updated Target Vocab Size (SentencePiece): 16000


In [129]:
import torch

# Hyperparams (can be adjusted for better performance, but keeping similar for now)
d_model = 128
num_heads = 4
d_ff = 256
num_layers = 2
dropout = 0.1 # Re-introduce dropout

model_spm = TinyTransformerSeq2SeqMT(
    src_vocab=src_vocab_size_spm,
    tgt_vocab=tgt_vocab_size_spm,
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    num_layers=num_layers,
    dropout=dropout
).to(device)

opt_spm = torch.optim.AdamW(model_spm.parameters(), lr=3e-4)
loss_fn_spm = nn.CrossEntropyLoss(ignore_index=pad_id)

# Training loop for the SentencePiece-enabled model
def run_epoch_spm(loader, train=True):
    model_spm.train() if train else model_spm.eval()
    total_loss = 0.0
    n_batches = 0

    for src, tgt_in, tgt_out in loader:
        src_key_mask = (src != pad_id)      # (B,S)
        tgt_key_mask = (tgt_in != pad_id)   # (B,T)

        logits, _, _ = model_spm(src, tgt_in, src_key_mask, tgt_key_mask)
        loss = loss_fn_spm(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

        if train:
            opt_spm.zero_grad(set_to_none=True)
            loss.backward()
            opt_spm.step()

        total_loss += loss.item()
        n_batches += 1

    return total_loss / max(1, n_batches)

print("Starting training with SentencePiece tokenization...")
# Train for a few epochs
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    train_loss_spm = run_epoch_spm(train_loader_spm, train=True)
    val_loss_spm = run_epoch_spm(val_loader_spm, train=False)
    print(f"Epoch {epoch:2d} | Train Loss: {train_loss_spm:.4f} | Val Loss: {val_loss_spm:.4f}")

Starting training with SentencePiece tokenization...


TypeError: EncoderBlockPostNorm.forward() got an unexpected keyword argument 'enc_mask'

In [ ]:
import sentencepiece as spm

# Load the trained SentencePiece models
sp_de = spm.SentencePieceProcessor(model_file='de_sp.model')
sp_en = spm.SentencePieceProcessor(model_file='en_sp.model')

# Define special token IDs based on SentencePiece defaults
# SentencePiece typically uses 0 for <unk>, 1 for <s>, 2 for </s>
# We will use 0 for padding as well, a common practice where padding takes the <unk> ID
pad_id = sp_en.unk_id() # Use <unk> id for padding
bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
unk_id = sp_en.unk_id()

print(f"SentencePiece: pad_id={pad_id}, bos_id={bos_id}, eos_id={eos_id}, unk_id={unk_id}")

In [ ]:
# New encode function using SentencePiece
def encode_spm(text, sp_model, max_len):
    # sp_model.encode_as_ids adds <s> and </s> by default if not suppressed
    ids = sp_model.encode_as_ids(text)

    # Ensure BOS and EOS are correctly handled.
    # SentencePiece encode_as_ids usually includes <s> and </s> by default.
    # We need to make sure they match our expected bos_id and eos_id consistently.
    # sp_model.encode_as_ids(text, add_bos=True, add_eos=True) is the default behavior

    # Clip sequence to max_len (including BOS/EOS if present)
    if len(ids) > max_len:
        # Preserve BOS and EOS if they are automatically added, or handle manually
        # Assuming sp_model.encode_as_ids already added them, we clip from the middle if necessary
        # For simplicity, if length exceeds max_len, we just truncate, ensuring BOS/EOS are at ends
        # A more robust solution might ensure BOS/EOS are always present after clipping
        ids = ids[:max_len-1] + [sp_model.eos_id()] # Truncate and ensure EOS
    elif len(ids) < max_len and ids[-1] != sp_model.eos_id():
        # Add EOS if not present (should be by default, but double check)
        ids.append(sp_model.eos_id())

    return ids

# Updated collate_fn to use SentencePiece
def collate_fn_spm(batch, max_src_len=64, max_tgt_len=64):
    src_ids_list = []
    tgt_in_list = []
    tgt_out_list = []

    for ex in batch:
        src_text = ex["translation"]["de"]
        tgt_text = ex["translation"]["en"]

        # Encode using SentencePiece
        src_ids = encode_spm(src_text, sp_de, max_src_len) # Contains <s>...</s>
        tgt_ids = encode_spm(tgt_text, sp_en, max_tgt_len) # Contains <s>...</s>

        # Teacher forcing:
        # tgt_in  = [BOS, w1, w2, ...]
        # tgt_out = [w1,  w2, ... , EOS]
        # SentencePiece encode_as_ids includes <s> and </s>.
        # We need to adjust for tgt_in/tgt_out if sp_model.encode_as_ids adds them.
        # Default behavior: sp_model.encode_as_ids(text) results in [<s>, tok1, tok2, ..., tokN, </s>]
        # then for training, tgt_in should be [<s>, tok1, ..., tokN]
        # and tgt_out should be [tok1, ..., tokN, </s>]
        tgt_in  = tgt_ids[:-1] # Remove the final </s> from tgt_ids for input
        tgt_out = tgt_ids[1:]  # Remove the initial <s> from tgt_ids for target

        src_ids_list.append(src_ids)
        tgt_in_list.append(tgt_in)
        tgt_out_list.append(tgt_out)

    def pad_to_max(seqs, pad_value):
        max_len = max(len(s) for s in seqs)
        out = torch.full((len(seqs), max_len), pad_value, dtype=torch.long)
        for i,s in enumerate(seqs):
            out[i, :len(s)] = torch.tensor(s, dtype=torch.long)
        return out

    src = pad_to_max(src_ids_list, pad_id)       # (B, S)
    tgt_in = pad_to_max(tgt_in_list, pad_id)     # (B, T)
    tgt_out = pad_to_max(tgt_out_list, pad_id)   # (B, T)

    return src.to(device), tgt_in.to(device), tgt_out.to(device)

# Re-create DataLoaders with the new collate_fn_spm
train_loader_spm = DataLoader(train_slice, batch_size=64, shuffle=True, collate_fn=lambda b: collate_fn_spm(b, max_src_len=64, max_tgt_len=64))
val_loader_spm = DataLoader(ds["validation"], batch_size=64, shuffle=False, collate_fn=lambda b: collate_fn_spm(b, max_src_len=64, max_tgt_len=64))

# Get updated vocabulary sizes
src_vocab_size_spm = sp_de.get_piece_size()
tgt_vocab_size_spm = sp_en.get_piece_size()

print(f"Updated Source Vocab Size (SentencePiece): {src_vocab_size_spm}")
print(f"Updated Target Vocab Size (SentencePiece): {tgt_vocab_size_spm}")

In [ ]:
import torch

# Hyperparams (can be adjusted for better performance, but keeping similar for now)
d_model = 128
num_heads = 4
d_ff = 256
num_layers = 2
dropout = 0.1 # Re-introduce dropout

model_spm = TinyTransformerSeq2SeqMT(
    src_vocab=src_vocab_size_spm,
    tgt_vocab=tgt_vocab_size_spm,
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    num_layers=num_layers,
    dropout=dropout
).to(device)

opt_spm = torch.optim.AdamW(model_spm.parameters(), lr=3e-4)
loss_fn_spm = nn.CrossEntropyLoss(ignore_index=pad_id)

# Training loop for the SentencePiece-enabled model
def run_epoch_spm(loader, train=True):
    model_spm.train() if train else model_spm.eval()
    total_loss = 0.0
    n_batches = 0

    for src, tgt_in, tgt_out in loader:
        src_key_mask = (src != pad_id)      # (B,S)
        tgt_key_mask = (tgt_in != pad_id)   # (B,T)

        logits, _, _ = model_spm(src, tgt_in, src_key_mask, tgt_key_mask)
        loss = loss_fn_spm(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))

        if train:
            opt_spm.zero_grad(set_to_none=True)
            loss.backward()
            opt_spm.step()

        total_loss += loss.item()
        n_batches += 1

    return total_loss / max(1, n_batches)

print("Starting training with SentencePiece tokenization...")
# Train for a few epochs
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    train_loss_spm = run_epoch_spm(train_loader_spm, train=True)
    val_loss_spm = run_epoch_spm(val_loader_spm, train=False)
    print(f"Epoch {epoch:2d} | Train Loss: {train_loss_spm:.4f} | Val Loss: {val_loss_spm:.4f}")

**Reasoning**:
The next step is to extract German sentences from the `train` split of the dataset and save them to a temporary file, with each sentence on a new line, as per the second instruction.



In [ ]:
import os

de_sentences = [example["translation"]["de"] for example in ds["train"]]
with open("de_sentences.txt", "w", encoding="utf-8") as f:
    for sentence in de_sentences:
        f.write(sentence + "\n")

print("German sentences extracted and saved to 'de_sentences.txt'.")

**Reasoning**:
Now that the German sentences are saved, I will train the SentencePiece model for German using the specified parameters, as per the third instruction.



In [124]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    '--input=de_sentences.txt --model_prefix=de_sp --vocab_size=16000 --model_type=bpe'
)

print("SentencePiece model for German (de_sp.model, de_sp.vocab) trained.")

SentencePiece model for German (de_sp.model, de_sp.vocab) trained.


In [125]:
en_sentences = [example["translation"]["en"] for example in ds["train"]]
with open("en_sentences.txt", "w", encoding="utf-8") as f:
    for sentence in en_sentences:
        f.write(sentence + "\n")

print("English sentences extracted and saved to 'en_sentences.txt'.")

English sentences extracted and saved to 'en_sentences.txt'.


**Reasoning**:
Now that the English sentences are saved, I will train the SentencePiece model for English using the specified parameters, as per the third instruction.

In [126]:
spm.SentencePieceTrainer.train(
    '--input=en_sentences.txt --model_prefix=en_sp --vocab_size=16000 --model_type=bpe'
)

print("SentencePiece model for English (en_sp.model, en_sp.vocab) trained.")

SentencePiece model for English (en_sp.model, en_sp.vocab) trained.
